# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [5]:
from google.colab import files
files.upload()
!ls

Saving cal_housing_clean.csv to cal_housing_clean (1).csv
'cal_housing_clean (1).csv'   landata-states.csv	  zecon.zip
 cal_housing_clean.csv	      pima-indians-diabetes.csv
 diabetes.csv		      sample_data


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [2]:
dataset = pd.read_csv("cal_housing_clean (1).csv")
dataset.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [3]:
dataset.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [0]:
x_data = dataset.drop('medianHouseValue', axis = 1)
labels = dataset['medianHouseValue']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels,
                                                    test_size = 0.3,
                                                    random_state= 101)

In [7]:
X_train.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'],
      dtype='object')

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
scaler = MinMaxScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [10]:
X_train = pd.DataFrame(np.array(scaled_X_train), 
                       columns = ['housingMedianAge', 'totalRooms', 
                                  'totalBedrooms', 'population',
                                  'households', 'medianIncome'],
                       index = X_train.index)
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [11]:
X_test = pd.DataFrame(np.array(scaled_X_test), 
                      columns = ['housingMedianAge', 'totalRooms', 
                                  'totalBedrooms', 'population',
                                  'households', 'medianIncome'],
                      index = X_test.index)
X_test.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,0.686275,0.046264,0.045158,0.025873,0.048841,0.353133
8816,0.705882,0.027417,0.020795,0.012709,0.023187,0.770182
7175,0.901961,0.032326,0.040813,0.041662,0.042592,0.133626
16714,0.313725,0.043212,0.046089,0.032840,0.048018,0.263576
14491,0.411765,0.088433,0.069367,0.043728,0.072192,0.660046


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [12]:
dataset.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [0]:
housing_age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')

In [0]:
feat_cols = [housing_age, rooms, bedrooms, population, medianIncome]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train, 
                                                 y_train,
                                                 batch_size = 10,
                                                 num_epochs = 1000,
                                                 shuffle = True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [23]:
dnn_model = tf.estimator.DNNRegressor(feature_columns= feat_cols,
                                      hidden_units = [6, 6, 6])

W0716 16:29:29.223546 139777028687744 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpfphn175x


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [24]:
dnn_model.train(input_fn = input_func,
                steps = 20000)

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [0]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(X_test, 
                                                      batch_size = 10,
                                                      shuffle = False)

In [0]:
predicted_val = dnn_model.predict(input_fn = pred_input_func)

In [0]:
final_preds = []

for pred in predicted_val:
  final_preds.append(pred['predictions'])

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [0]:
from sklearn.metrics import mean_squared_error

In [29]:
mean_squared_error(y_test, final_preds) ** 0.5

99001.95305717985

# Great Job!